# Step1:导入相关包

In [ ]:
from datasets import Dataset,load_dataset,load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer,BitsAndBytesConfig

In [ ]:
import wandb 
wandb.login(key="1b8adc705fb9b3e125c05f15107ad7c22c830811")
wandb.init(project="deepseek_r1_distill_llama_8b_qlora_4bit")

# Step2 加载数据集

In [ ]:
ds = load_from_disk("../data/alpaca_data_zh/")
ds

In [ ]:
ds[:3]

#  Step3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("D:/study/LLM/train/models/DeepSeek-R1-Distill-Llama-8B")

tokenizer

In [6]:
tokenizer.padding_side = "right"  # 一定要设置padding_side为right，否则batch大于1时可能不收敛

In [7]:
tokenizer.pad_token_id = 2

In [8]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
print(tokenized_ds[0]["input_ids"])

# Step4 创建模型

In [ ]:
import torch
# 多卡情况，可以去掉device_map="auto"，否则会将模型拆开
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype=torch.bfloat16,  
    bnb_4bit_quant_type="nf4",  
    bnb_4bit_use_double_quant=True  
)
model = AutoModelForCausalLM.from_pretrained(
    "D:/study/LLM/train/models/DeepSeek-R1-Distill-Llama-8B",
    quantization_config=bnb_config,  # 这里替换 load_in_4bit
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
model

In [ ]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("红楼梦的《好了歌》是怎么说的", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

In [ ]:
model.config

# Lora

# PEFT Step1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

# PEFT Step2 创建模型

In [17]:
model = get_peft_model(model, config)

In [ ]:
model

In [ ]:
config

In [20]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
model.print_trainable_parameters()

# Step5 配置训练参数

In [22]:
args = TrainingArguments(
    output_dir="../output",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="../logs",
    logging_steps=10,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    report_to="wandb", 
)

# Step6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

# Step7 模型训练

In [ ]:
trainer.train()

# Step8 模型推理

In [ ]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("红楼梦的好了歌怎么说", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

In [ ]:
model.merge_and_unload() 

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

# 模型保存

In [ ]:
# 指定保存路径
save_path = "D:/study/LLM/train/output/DeepSeek-R1-Distill-Llama-8B-merged"

# 确保 merge_and_unload() 之后，模型已经回到了 FP16 或者 FP32
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
